In [1]:
# Importing Dependencies
import os
from bs4 import BeautifulSoup as bs
import requests
from flask import Flask, jsonify
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
url = 'https://www.e-chords.com/chords'

response = requests.get(url)

soup = bs(response.text, 'html.parser')
songs = soup.find_all('div', class_='track')

data = []
for song in songs:

    title = song.find('p', class_='nome-musica').text
    artist = song.find('p', class_='nome-artista').text
    var = {'title': title, 'artist': artist}
    data.append([var])

print(data)

[[{'title': 'Shallow', 'artist': 'Lady Gaga'}], [{'title': 'Wonderwall', 'artist': 'Oasis'}], [{'title': 'Hotel California (Acoustic)', 'artist': 'Eagles'}], [{'title': 'Let It Be', 'artist': 'The Beatles'}], [{'title': 'Wish You Were Here', 'artist': 'Pink Floyd'}], [{'title': 'Take Me Home, Country Roads (ver. 2)', 'artist': 'John Denver'}], [{'title': 'Yesterday', 'artist': 'The Beatles'}], [{'title': 'Stand by me', 'artist': 'Ben E. King'}], [{'title': 'Creep', 'artist': 'Radiohead'}], [{'title': 'All Of Me', 'artist': 'John Legend'}], [{'title': 'Have You Ever Seen The Rain', 'artist': 'Creedence Clearwater Revival'}], [{'title': 'Here Comes The Sun', 'artist': 'The Beatles'}], [{'title': 'Your Song', 'artist': 'Elton John'}], [{'title': 'Stairway to Heaven', 'artist': 'Led Zeppelin'}], [{'title': 'Afterglow', 'artist': 'Ed Sheeran'}], [{'title': 'Hey Jude', 'artist': 'The Beatles'}], [{'title': 'Somewhere Over The Rainbow', 'artist': "Israel Kamakawiwo'ole"}], [{'title': 'Always 

In [3]:
def urlify(the_string):
    url_friendly = the_string.replace(" ", "-")
    url_friendly = url_friendly.replace(".", "")
    url_friendly = url_friendly.replace("(", "")
    url_friendly = url_friendly.replace(")", "")
    url_friendly = url_friendly.replace(",", "")
    url_friendly = url_friendly.replace("'", "")
    url_friendly = url_friendly.lower()
    return(url_friendly)

In [4]:
all_chords = []
for song in data:
    new_url = 'https://www.e-chords.com/chords' + '/' + urlify(song[0]['artist']) + '/' + urlify(song[0]['title'])

    response = requests.get(new_url)
    soup = bs(response.text, 'html.parser')

    core = soup.find('pre', class_='core')
    
    try:
        chords_html = core.find_all('u')
        chords = []
        for chord in chords_html:
            chords.append(chord.text)

        for i in range(0, len(chords)):
            prog = chords[i:i+4]
            all_chords.append(prog)

    except:
        print('Error '+ song[0]['title'])
    print('Processing ' + song[0]['title'])

Processing Shallow
Processing Wonderwall
Processing Hotel California (Acoustic)
Processing Let It Be
Processing Wish You Were Here
Processing Take Me Home, Country Roads (ver. 2)
Processing Yesterday
Processing Stand by me
Processing Creep
Processing All Of Me
Processing Have You Ever Seen The Rain
Processing Here Comes The Sun
Processing Your Song
Processing Stairway to Heaven
Processing Afterglow
Processing Hey Jude
Processing Somewhere Over The Rainbow
Processing Always Remember Us This Way
Processing Zombie
Processing The House Of The Rising Sun
Processing Nothing Else Matters
Processing La Vie En Rose
Processing What's Up
Processing Snowman
Processing Something
Processing Perfect
Processing Love of my Life
Processing Sultans of Swing
Processing Wonderful Tonight
Processing Losing my religion
Processing Tears In Heaven (acoustic)
Processing Sweet Child O' Mine
Processing Bohemian Rhapsody
Processing Father and Son
Processing While My Guitar Gently Weeps
Processing Can't Help Fallin

In [5]:
df = pd.DataFrame(all_chords)
df

,0,1,2,3
0,Em7,D/F#,G,C
1,D/F#,G,C,G
2,G,C,G,D
3,C,G,D,Em7
4,G,D,Em7,D/F#
...,...,...,...,...
8588,Am,Am/G,F,F
8589,Am/G,F,F,C
8590,F,F,C,None
8591,F,C,None,None


In [6]:
chord_list = [i for i in all_chords if i] 
len(chord_list)

8593

In [7]:
flatten = lambda t: [item for sublist in t for item in sublist]
flat = flatten(chord_list)
len(flat)

33790

In [8]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
flat = np.asarray(flat).reshape(-1,1)

enc.fit(flat)
flat_enc = enc.transform(flat)

enc_chords = []
for i in range(8593):
    enc_chords.append(flat_enc[i*4:i*4+4])
# for i in flat_enc:
#     temp_arr = []
#     for j in range(i,i+4):
#         temp_arr.push(flat_enc[j])
#     enc_chords.push(temp_arr)


C:\Users\dpatc\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [9]:
max1 = 0
for row in enc_chords:
    for i in row:
        if i > max1:
            max1 = i
            
max1

188

In [10]:
# print(enc_chords)
# for i in enc_chords:
#     print(i)
decoder = lambda t: [enc.inverse_transform([item]) for sublist in t for item in sublist]
decoded = decoder(enc_chords)

In [11]:
len(enc_chords)

8593

In [12]:
enc_chords = [i for i in enc_chords if i.any] 
len(enc_chords)

8593

In [13]:
from sklearn.model_selection import train_test_split

X = []
y = []
i=0
for whatever in enc_chords:
    if len(whatever) == 0:
        i = i + 1
    X.append(whatever[0:3])
    y.append(whatever[3:4])
    
# y = y.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(len(X_test))
X_train = [i for i in X_train if len(i) > 2]
X_test = [i for i in X_test if len(i) > 2] 
y_train = [i for i in y_train if len(i) > 0]
y_test = [i for i in y_test if len(i) > 0]
# X_train

2149


In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=150, activation='sigmoid', input_dim=3))
model.add(Dense(units=200, activation='sigmoid'))
model.add(Dense(units=189, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 150)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               30200     
_________________________________________________________________
dense_2 (Dense)              (None, 189)               37989     
Total params: 68,789
Trainable params: 68,789
Non-trainable params: 0
_________________________________________________________________


In [16]:
import tensorflow as tf

y_train = tf.keras.utils.to_categorical(y_train)

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [19]:


X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.int64) 
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.int64) 
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.int64) 
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.int64) 

print(y_train.shape)

(6340, 189)


In [20]:
model.fit(X_train_tensor, y_train_tensor, epochs=175, shuffle=True, verbose=2)

Epoch 1/175
199/199 - 0s - loss: 3.3859 - accuracy: 0.2257
Epoch 2/175
199/199 - 0s - loss: 3.1218 - accuracy: 0.2797
Epoch 3/175
199/199 - 0s - loss: 3.0257 - accuracy: 0.2998
Epoch 4/175
199/199 - 0s - loss: 2.9456 - accuracy: 0.3062
Epoch 5/175
199/199 - 0s - loss: 2.8805 - accuracy: 0.3181
Epoch 6/175
199/199 - 0s - loss: 2.8337 - accuracy: 0.3121
Epoch 7/175
199/199 - 0s - loss: 2.7886 - accuracy: 0.3156
Epoch 8/175
199/199 - 0s - loss: 2.7495 - accuracy: 0.3235
Epoch 9/175
199/199 - 0s - loss: 2.7109 - accuracy: 0.3284
Epoch 10/175
199/199 - 0s - loss: 2.6799 - accuracy: 0.3319
Epoch 11/175
199/199 - 0s - loss: 2.6509 - accuracy: 0.3358
Epoch 12/175
199/199 - 0s - loss: 2.6137 - accuracy: 0.3413
Epoch 13/175
199/199 - 0s - loss: 2.5910 - accuracy: 0.3412
Epoch 14/175
199/199 - 0s - loss: 2.5668 - accuracy: 0.3461
Epoch 15/175
199/199 - 0s - loss: 2.5429 - accuracy: 0.3470
Epoch 16/175
199/199 - 0s - loss: 2.5231 - accuracy: 0.3500
Epoch 17/175
199/199 - 0s - loss: 2.4941 - accura

199/199 - 0s - loss: 1.4253 - accuracy: 0.5442
Epoch 138/175
199/199 - 0s - loss: 1.4204 - accuracy: 0.5498
Epoch 139/175
199/199 - 0s - loss: 1.4223 - accuracy: 0.5487
Epoch 140/175
199/199 - 0s - loss: 1.4163 - accuracy: 0.5528
Epoch 141/175
199/199 - 0s - loss: 1.4144 - accuracy: 0.5517
Epoch 142/175
199/199 - 0s - loss: 1.4131 - accuracy: 0.5498
Epoch 143/175
199/199 - 0s - loss: 1.4077 - accuracy: 0.5511
Epoch 144/175
199/199 - 0s - loss: 1.4025 - accuracy: 0.5525
Epoch 145/175
199/199 - 0s - loss: 1.3941 - accuracy: 0.5565
Epoch 146/175
199/199 - 0s - loss: 1.3945 - accuracy: 0.5571
Epoch 147/175
199/199 - 0s - loss: 1.3941 - accuracy: 0.5502
Epoch 148/175
199/199 - 0s - loss: 1.3831 - accuracy: 0.5580
Epoch 149/175
199/199 - 0s - loss: 1.3780 - accuracy: 0.5568
Epoch 150/175
199/199 - 0s - loss: 1.3721 - accuracy: 0.5617
Epoch 151/175
199/199 - 0s - loss: 1.3738 - accuracy: 0.5579
Epoch 152/175
199/199 - 0s - loss: 1.3681 - accuracy: 0.5625
Epoch 153/175
199/199 - 0s - loss: 1.3

In [22]:

predictions = model.predict(tf.expand_dims(enc.transform(['G','A','D']), axis=0))

predictions

array([[4.35031857e-03, 1.16748489e-18, 1.90204173e-05, 1.45410656e-10,
        9.18444867e-26, 4.47108363e-07, 1.31450332e-17, 1.93571182e-17,
        1.76584781e-05, 1.77979300e-12, 2.77505251e-07, 1.97757717e-12,
        8.25013994e-05, 2.43808155e-31, 7.96043006e-12, 5.47819070e-07,
        7.02063407e-12, 1.93060791e-18, 4.20027333e-15, 1.23770815e-05,
        7.68154857e-16, 5.73260915e-20, 1.81678783e-19, 9.60669033e-27,
        6.01416250e-10, 9.08595213e-11, 2.25703556e-22, 4.66316791e-23,
        2.07436213e-04, 1.12450087e-23, 3.59822371e-31, 4.73645645e-12,
        1.43648613e-25, 7.32970474e-15, 4.31571476e-18, 2.86009381e-05,
        7.33318318e-27, 5.87767820e-33, 2.89839809e-05, 1.69537814e-19,
        1.13931154e-22, 2.64862507e-07, 3.76294139e-18, 2.31611204e-28,
        2.56558298e-04, 1.85112192e-09, 5.91203925e-14, 9.83564714e-24,
        3.61962345e-20, 5.69460863e-05, 3.98835084e-20, 3.68528211e-12,
        4.53693593e-12, 8.50692737e-21, 2.55712983e-20, 2.166732

In [23]:
guesses = predictions[0].argsort()[::-1][0:5]
guesses

array([166, 145, 137, 139,  61], dtype=int64)

In [24]:
guess_score = predictions[0].sort()
prediction_score = sorted(predictions[0],reverse=True)[0:5]
prediction_score

[0.78244823, 0.14141482, 0.024558477, 0.022429032, 0.0069315843]

In [25]:
prediction = enc.inverse_transform(guesses)
prediction

array(['G', 'F#m', 'Em', 'Em7', 'C'], dtype='<U11')

In [26]:
for i in range(0,len(prediction)):
    print(f'{prediction[i]}: {prediction_score[i]} certainty')

G: 0.7824482321739197 certainty
F#m: 0.1414148211479187 certainty
Em: 0.024558477103710175 certainty
Em7: 0.022429032251238823 certainty
C: 0.00693158432841301 certainty


In [27]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [28]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [29]:
np.save('classes.npy', enc.classes_)